# Advent of Code 2022 - Day 9

In [1]:
from pathlib import Path
from typing import Final, List, Dict, Tuple, Set, Any, TypeAlias, Optional

## Part One

- [X] Read input
- [ ] Split each instruction to a Tuple of (Direction, Number)
- [ ] Create a grid for the rope to move on?
- [ ] Move Head
- [ ] Compare Tail position to head and move accordingly
- [ ] Mark position of tail on a boolean grid (True is tail has passed that space)
- [ ] Count grid


In [2]:
INPUT_NAME : str = "day9_input_test.txt"
MovePosition : TypeAlias = Tuple[int, int]

In [3]:
def read_input(input_name: str) -> List[str]:
    input_path : Path = Path.cwd() / "input" / input_name

    with open(input_path, "r") as input_file:

        lines : List = input_file.readlines()

    return [line.rstrip("\n") for line in lines]

In [4]:
instructions_input : List[MovePosition]= [tuple(line.split(" ")) for line in read_input(INPUT_NAME)]

In [5]:

STARTING_POSITION : Final[MovePosition]= (0,0)
STEP = 1
ROW = 0
COL = 1
DIAGONAL = "/"
DIRECTION = 0
COUNT = 1

In [6]:
all_covered_positions : Set[MovePosition] = set([])

In [7]:
def make_step_move(current_position: MovePosition, move: MovePosition, is_tail: bool = False) -> MovePosition:
    """
    Co-ordinates are relative to start (0,0), as (row, col)
    """  
    
    return (current_position[ROW] + move[ROW], current_position[COL] + move[COL])
    

In [8]:
def move_head(current_position: MovePosition, move: MovePosition) -> MovePosition:
    
    return make_step_move(current_position, move)

In [39]:
def move_tail(direction: str, head_position: MovePosition, current_position: MovePosition) -> MovePosition:
    """
    Delta position: 
        -ve means below or to the left of head
    
    Diagonal movement:
        Head movmement that results in a diagonal delta should result in no tail movement.
        Next tail movement should then be to where head was in previous.
    """
    
    delta_position = (current_position[ROW]-head_position[ROW], current_position[COL]-head_position[COL])
    print(f"Delta: {delta_position}")
    
    # Check if in same row/col (else diagonal move)
    # Both checks will be false if diagonal move required
    if max(abs(delta_position[ROW]), abs(delta_position[COL])) < 2: # Head on top of tail! (would only be if U then D, or L then R)
        print("Returning current")
        return current_position
    
    elif delta_position[ROW] == head_position[ROW] or delta_position[COL] == head_position[COL]:
        # Move orthogonally
        move = generate_step_move(direction)
        print("Orthogonal")
        return make_step_move(current_position, move)
    else:
        # Move diagonally
        if max(delta_position) < 2:
        # If delta position is (1,1) then don't move
            print("Diagonal, no move")
            return current_position

        else:
        # If |delta position| is (2,1) or (1,2) then move diagonally
            print("Diagonal, move")
            move = generate_step_move(DIAGONAL, delta_position)
            return make_step_move(current_position, move)

In [40]:
def generate_step_move(direction: str, diagonal_delta: Optional[MovePosition] = None) -> MovePosition:
    
    step = -1 if direction in ["L", "D"] else 1
    
    if direction in ["L", "R"]:
        move : MovePosition = (0, step)
    elif direction in ["U", "D"]:
        move = (step, 0)
    elif direction == DIAGONAL:
        
        def check_diag_move(num_spaces: int) -> int:
            """
                If a row or col move is > 2 it is diagonal
                Function converts this to a tail move
            """
            
            if num_spaces > 1:
                if 1-(num_spaces<0):    # num_spaces is positive
                    return num_spaces -1
                else:                   # num_spaces is negative
                    return num_spaces + 1
            else:
                return num_spaces
            
        move = (check_diag_move(diagonal_delta[ROW]), check_diag_move(diagonal_delta[COL]))
        
    return move

In [41]:
def make_move(
    direction: str, count: str, head_position: MovePosition, tail_position: MovePosition
) -> Tuple[MovePosition, MovePosition, List[MovePosition]]:
    """
    Returns current head position, and list of positions tail has been in for this move
    """

    count = int(count)

    current_head_position : MovePosition = head_position
    current_tail_position : MovePosition = tail_position
    covered_positions : List = []

    for idx in range(count):
        start_pos : bool = current_head_position == STARTING_POSITION
        move : MovePosition = generate_step_move(direction)
        current_head_position = move_head(current_head_position, move)
        if not start_pos:
            print("Moving tail!")
            current_tail_position = move_tail(direction, current_head_position, current_tail_position)
            covered_positions.append(current_tail_position)
        print(f"Head: {current_head_position}, Tail: {current_tail_position}")
        
    return current_head_position, current_tail_position, covered_positions
        

In [42]:
# position_set.update(make_move("U", 4, STARTING_POSITION, STARTING_POSITION)[2])

In [43]:
for idx, instruction in enumerate(instructions_input):

    if idx == 0:
        current_head_position, current_tail_position = (
            STARTING_POSITION,
            STARTING_POSITION,
        )
        
    print(f"[{idx}]")


    current_head_position, current_tail_position, covered_positions = make_move(
        instruction[DIRECTION],
        instruction[COUNT],
        current_head_position,
        current_tail_position,
    )
    
    all_covered_positions.update(covered_positions)
    
    
print(len(all_covered_positions))

[0]
Head: (0, 1), Tail: (0, 0)
Moving tail!
Delta: (0, -2)
Orthogonal
Head: (0, 2), Tail: (0, 1)
Moving tail!
Delta: (0, -2)
Orthogonal
Head: (0, 3), Tail: (0, 2)
Moving tail!
Delta: (0, -2)
Orthogonal
Head: (0, 4), Tail: (0, 3)
[1]
Moving tail!
Delta: (-1, -1)
Returning current
Head: (1, 4), Tail: (0, 3)
Moving tail!
Delta: (-2, -1)
Diagonal, no move
Head: (2, 4), Tail: (0, 3)
Moving tail!
Delta: (-3, -1)
Diagonal, no move
Head: (3, 4), Tail: (0, 3)
Moving tail!
Delta: (-4, -1)
Diagonal, no move
Head: (4, 4), Tail: (0, 3)
[2]
Moving tail!
Delta: (-4, 0)
Diagonal, no move
Head: (4, 3), Tail: (0, 3)
Moving tail!
Delta: (-4, 1)
Diagonal, no move
Head: (4, 2), Tail: (0, 3)
Moving tail!
Delta: (-4, 2)
Diagonal, move
Head: (4, 1), Tail: (-4, 4)
[3]
Moving tail!
Delta: (-7, 3)
Diagonal, move
Head: (3, 1), Tail: (-11, 6)
[4]
Moving tail!
Delta: (-14, 4)
Diagonal, move
Head: (3, 2), Tail: (-25, 9)
Moving tail!
Delta: (-28, 6)
Diagonal, move
Head: (3, 3), Tail: (-53, 14)
Moving tail!
Delta: (-5